In [14]:
%matplotlib inline
from datasets import load_from_disk
from pyvenn import *

## 加载验证数据集

In [6]:
def load_validation_dataset(dataset_name):
    dataset=load_from_disk(f'../defects4j_validation/dataset_validated/{dataset_name}')
    print(f' \n==========\n{dataset_name}\n==========\n',dataset)
    return dataset

In [7]:
codellama_vanilla=load_validation_dataset('codellama_vanilla')
codellama_classinfo=load_validation_dataset('codellama_classinfo')
codellama_classinfo_lora=load_validation_dataset('codellama_classinfo_lora')
codellama_no_classinfo_lora=load_validation_dataset('codellama_no_classinfo_lora')
repairllama=load_validation_dataset('repairllama')
repairllama_classinfo=load_validation_dataset('repairllama_classinfo')
repairllama_classinfo_lora=load_validation_dataset('')

 
codellama_vanilla
 Dataset({
    features: ['methodInformation', 'involvedTypesInformation', 'filePath', 'classInformation', 'buggyInfo', 'projectName', 'bug_id', 'start_line', 'end_line', 'path', 'fix_code', 'pre_context', 'post_context', 'buggy_code', 'input', 'gen', 'test_res'],
    num_rows: 479
})
 
codellama_classinfo
 Dataset({
    features: ['methodInformation', 'involvedTypesInformation', 'filePath', 'classInformation', 'buggyInfo', 'projectName', 'bug_id', 'start_line', 'end_line', 'path', 'fix_code', 'pre_context', 'post_context', 'buggy_code', 'input', 'gen', 'test_res'],
    num_rows: 479
})
 
codellama_classinfo_lora
 Dataset({
    features: ['methodInformation', 'involvedTypesInformation', 'filePath', 'classInformation', 'buggyInfo', 'projectName', 'bug_id', 'start_line', 'end_line', 'path', 'fix_code', 'pre_context', 'post_context', 'buggy_code', 'input', 'gen', 'test_res'],
    num_rows: 479
})
 
codellama_no_classinfo_lora
 Dataset({
    features: ['methodInformatio

In [8]:
dataset_dict={
    'codellama_vanilla':codellama_vanilla,
    'codellama_classinfo':codellama_classinfo,
    'codellama_classinfo_lora':codellama_classinfo_lora,
    'codellama_no_classinfo_lora':codellama_no_classinfo_lora,
    'repairllama':repairllama,
    'repairllama_classinfo':repairllama_classinfo,
    'repairllama_classinfo_lora':repairllama_classinfo_lora,
}

In [ ]:
print(" public static Locale toLocale(final String str) {\n        if (str == null) {\n            return null;\n        }\n        final int len = str.length();\n        if (len < 2) {\n            throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n        }\n        final char ch0 = str.charAt(0);\n// buggy code\n//             final char ch1 = str.charAt(1);\n//             if (!Character.isLowerCase(ch0) || !Character.isLowerCase(ch1)) {\n//                 throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n//             }\n//             if (len == 2) {\n//                 return new Locale(str);\n//             }\n//             if (len < 5) {\n//                 throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n//             }\n//             if (str.charAt(2) != '_') {\n//                 throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n//             }\n//             final char ch3 = str.charAt(3);\n//             if (ch3 == '_') {\n//                 return new Locale(str.substring(0, 2), \"\", str.substring(4));\n//             }\n//             final char ch4 = str.charAt(4);\n//             if (!Character.isUpperCase(ch3) || !Character.isUpperCase(ch4)) {\n//                 throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n//             }\n//             if (len == 5) {\n//                 return new Locale(str.substring(0, 2), str.substring(3, 5));\n//             }\n//             if (len < 7) {\n//                 throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n//             }\n//             if (str.charAt(5) != '_') {\n//                 throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n//             }\n//             return new Locale(str.substring(0, 2), str.substring(3, 5), str.substring(6));\n       final char ch1 = str.charAt(1);\n        if (!Character.isLowerCase(ch0) || !Character.isLowerCase(ch1)) {\n            throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n        }\n        if (len == 2) {\n            return new Locale(str);\n        }\n        if (len < 5) {\n            throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n        }\n        if (str.charAt(2) != '_') {\n            throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n        }\n        final char ch3 = str.charAt(3);\n        if (ch3 == '_') {\n            return new Locale(str.substring(0, 2), \"\", str.substring(4));\n        }\n        final char ch4 = str.charAt(4);\n        if (!Character.isUpperCase(ch3) || !Character.isUpperCase(ch4)) {\n            throw new IllegalArgumentException(\"Invalid locale format: \" + str);\n        }\n           }\n")

## 统计correctness

In [9]:
def determine_correctness(correctness_list):
    # 按优先级确定correctness
    if 'plausible' in correctness_list:
        return 'plausible'
    elif 'wrong' in correctness_list:
        return 'wrong'
    elif 'uncompilable' in correctness_list:
        return 'uncompilable'
    else:
        return 'timeout'  # 如果列表中没有已知的correctness值

def statistics_by_correctness(dataset):
    # 初始化一个字典来临时存储每个bug_id的所有correctness值
    temp_result = {}
    # 初始化最终结果字典
    final_result = {}

    # 收集每个bug_id的所有correctness值
    for row in dataset:
        bug_id = row['bug_id']
        test_res_list = row['test_res']  # 这是一个列表，不是单个字典
        if not test_res_list:
            continue
        if bug_id not in temp_result:
            temp_result[bug_id] = []
        # 遍历test_res_list中的每个字典
        for test_res in test_res_list:
            correctness = test_res['correctness']
            temp_result[bug_id].append(correctness)

    # 确定每个bug_id的最终correctness并组织最终结果
    for bug_id, correctness_list in temp_result.items():
        final_correctness = determine_correctness(correctness_list)
        if final_correctness not in final_result:
            final_result[final_correctness] = []
        final_result[final_correctness].append(bug_id)

    return final_result



In [23]:
res_dict={}
for name in dataset_dict:
    print(f'========={name}=========')
    dataset= dataset_dict[name]
    res=statistics_by_correctness(dataset)
    lst=['plausible','wrong','uncompilable']
    s=0
    for i in lst:
        print(f'{i}:', len(res[i]))
        s+=len(res[i])
    print('timeout or error', 479-s)
    res_dict[name]=res

=========codellama_vanilla=========
plausible: 116
wrong: 238
uncompilable: 119
timeout or error 6
=========codellama_classinfo=========
plausible: 87
wrong: 274
uncompilable: 113
timeout or error 5
=========codellama_classinfo_lora=========
plausible: 87
wrong: 284
uncompilable: 102
timeout or error 6
=========codellama_no_classinfo_lora=========
plausible: 86
wrong: 269
uncompilable: 119
timeout or error 5
=========repairllama=========
plausible: 114
wrong: 254
uncompilable: 106
timeout or error 5
=========repairllama_classinfo=========
plausible: 114
wrong: 255
uncompilable: 104
timeout or error 6
=========repairllama_classinfo_lora=========
plausible: 97
wrong: 261
uncompilable: 116
timeout or error 5


def count_correctness(dataset):
    # 初始化一个字典来计数不同correctness值的出现次数
    correctness_count = {}

    # 遍历数据集中的每一行
    for row in dataset:
        test_res_list = row['test_res']  # 获取当前bug_id的test_res列表
        if not test_res_list:
            continue
        # 遍历test_res_list中的每个字典
        for test_res in test_res_list:
            correctness = test_res['correctness']
            # 更新correctness值的出现次数
            if correctness in correctness_count:
                correctness_count[correctness] += 1
            else:
                correctness_count[correctness] = 1

    return correctness_count


# 调用函数并打印结果
correctness_count = count_correctness(repairllama_classinfo_lora)
print(correctness_count)


## unique bug_id

In [36]:
def find_unique_plausible_ids(res_dict):
    # 初始化一个字典来存储最终结果
    unique_plausible_ids = {dataset: [] for dataset in res_dict}

    # 遍历每个数据集，找到每个数据集中的plausible bug_id
    for dataset, correctness_dict in res_dict.items():
        plausible_ids = correctness_dict.get('plausible', [])
        
        # 对于每个plausible的bug_id，检查它是否在其他数据集中出现过
        for bug_id in plausible_ids:
            is_unique = True  # 假设当前bug_id是唯一的，直到证明它在其他数据集中出现过
            
            # 检查其他数据集
            for other_dataset, other_correctness_dict in res_dict.items():
                if dataset == other_dataset:
                    continue  # 跳过当前正在检查的数据集
                # 如果bug_id在其他数据集的任何correctness下出现，则不是唯一的
                if any(bug_id in ids for ids in other_correctness_dict['plausible']):
                    is_unique = False
                    break  # 一旦找到重复项，就停止检查当前bug_id
            
            # 如果当前bug_id是唯一的，则添加到结果字典中
            if is_unique:
                unique_plausible_ids[dataset].append(bug_id)

    return unique_plausible_ids


# 调用函数并打印结果
unique_plausible_ids = find_unique_plausible_ids(res_dict)
print(unique_plausible_ids)


{'codellama_vanilla': ['Csv-14', 'Csv-15', 'Compress-41', 'Chart-10', 'Closure-78', 'Mockito-22', 'Gson-18', 'Compress-26', 'Closure-58', 'Lang-37', 'Chart-24', 'Compress-32'], 'codellama_classinfo': ['JacksonXml-4', 'Lang-40'], 'codellama_classinfo_lora': ['Closure-109', 'Lang-38'], 'codellama_no_classinfo_lora': ['Closure-124', 'Jsoup-49'], 'repairllama': ['Compress-24', 'Closure-65', 'Time-18', 'Cli-37', 'Compress-44'], 'repairllama_classinfo': ['JacksonDatabind-24', 'Lang-16', 'JacksonDatabind-39', 'Lang-55'], 'repairllama_classinfo_lora': ['Math-105', 'Codec-2', 'Csv-5']}


In [ ]:
print('unique bug_id results')
for name in unique_plausible_ids:
    id_lst=unique_plausible_ids[name]
    dataset=dataset_dict[name]
    
    print(f'========={name}=========\n\n')
    for ids in id_lst:
        sample=dataset.filter(lambda x:x['bug_id']==ids)[0]
        print(f'------{ids}-------')
        print(sample['input'])
        print('fix: ', sample['fix_code'])
        for i in sample['test_res']
        print()
    